In [1]:
# 메모리 변수 모두 제거
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

In [3]:
import numpy as np
import pandas as pd

In [4]:
movie_data = pd.read_csv('C:/Workspace/분석데이터활용/1번째 프로젝트/data/TMDb_updated.csv', sep=',')
movie_data.head()

,Unnamed: 0,title,overview,original_language,vote_count,vote_average
0,0,Ad Astra,"The near future, a time when both hope and har...",en,2853,5.9
1,1,Bloodshot,"After he and his wife are murdered, marine Ray...",en,1349,7.2
2,2,Bad Boys for Life,Marcus and Mike are forced to confront new thr...,en,2530,7.1
3,3,Ant-Man,Armed with the astonishing ability to shrink i...,en,13611,7.1
4,4,Percy Jackson: Sea of Monsters,"In their quest to confront the ultimate evil, ...",en,3542,5.9


In [5]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         10000 non-null  int64  
 1   title              10000 non-null  object 
 2   overview           9970 non-null   object 
 3   original_language  10000 non-null  object 
 4   vote_count         10000 non-null  int64  
 5   vote_average       10000 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 468.9+ KB


In [8]:
movie_data.describe()

,vote_count,vote_average
count,10000.000000,10000.000000
mean,1020.825100,6.306230
std,1992.305005,1.354259
min,0.000000,0.000000
25%,143.000000,5.800000
50%,332.000000,6.500000
75%,926.250000,7.100000
max,25148.000000,10.000000


In [6]:
# 불필요한 Unnamed column 제거
movie_data.drop('Unnamed: 0',axis=1,inplace=True)
movie_data.head()

,title,overview,original_language,vote_count,vote_average
0,Ad Astra,"The near future, a time when both hope and har...",en,2853,5.9
1,Bloodshot,"After he and his wife are murdered, marine Ray...",en,1349,7.2
2,Bad Boys for Life,Marcus and Mike are forced to confront new thr...,en,2530,7.1
3,Ant-Man,Armed with the astonishing ability to shrink i...,en,13611,7.1
4,Percy Jackson: Sea of Monsters,"In their quest to confront the ultimate evil, ...",en,3542,5.9


In [7]:
# columns 확인
print(movie_data.columns)

Index(['title', 'overview', 'original_language', 'vote_count', 'vote_average'], dtype='object')


In [9]:
movie_data.isnull().sum()

title                 0
overview             30
original_language     0
vote_count            0
vote_average          0
dtype: int64

In [10]:
# nan값을 빈간으로 바꾸기
movie_data['overview'] = movie_data.overview.replace(np.NAN,'')

In [11]:
movie_data.isnull().sum()

title                0
overview             0
original_language    0
vote_count           0
vote_average         0
dtype: int64

In [12]:
md = movie_data
md.head()

,title,overview,original_language,vote_count,vote_average
0,Ad Astra,"The near future, a time when both hope and har...",en,2853,5.9
1,Bloodshot,"After he and his wife are murdered, marine Ray...",en,1349,7.2
2,Bad Boys for Life,Marcus and Mike are forced to confront new thr...,en,2530,7.1
3,Ant-Man,Armed with the astonishing ability to shrink i...,en,13611,7.1
4,Percy Jackson: Sea of Monsters,"In their quest to confront the ultimate evil, ...",en,3542,5.9


In [13]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C # 전체 영화 평균 점수

5.8732

In [14]:
# 상위 5%에 해당하는 vote_counts 값(평가에 사용할 최소한의 투표수)
m = vote_counts.quantile(0.95)
m

4462.149999999998

In [15]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title','overview','vote_count', 'vote_average']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(500, 4)

In [16]:
# 평가 점수 계산 함수
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [17]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1) # 평점

In [24]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [25]:
qualified.head(5) # 평점과 인기도 기준 추천 시스템

,title,overview,vote_count,vote_average,wr
80,Inception,"Cobb, a skilled thief who commits corporate es...",25148,8,7.679498
47,Interstellar,Interstellar chronicles the adventures of a gr...,21589,8,7.635713
88,The Dark Knight,Batman raises the stakes in his war on crime. ...,21561,8,7.635321
243,Fight Club,A ticking-time-bomb insomniac and a slippery s...,18664,8,7.589638
216,Pulp Fiction,"A burger-loving hit man, his philosophical par...",18003,8,7.577563


## 콘텐츠 기반 추천

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from ast import literal_eval

In [26]:
smd = movie_data

In [28]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['overview'])

In [29]:
tfidf_matrix.shape

(10000, 240943)

In [30]:
# 코사인 유사도
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [32]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

print(titles.head(), indices.head())

0                          Ad Astra
1                         Bloodshot
2                 Bad Boys for Life
3                           Ant-Man
4    Percy Jackson: Sea of Monsters
Name: title, dtype: object title
Ad Astra                          0
Bloodshot                         1
Bad Boys for Life                 2
Ant-Man                           3
Percy Jackson: Sea of Monsters    4
dtype: int64


In [33]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] # 상위 30개만 추천
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [34]:
get_recommendations('The Godfather').head(10) # 선택한 영화에 따른 추천 영화

233      The Godfather: Part II
7451                 Proud Mary
7814                 Blood Ties
9884                   Election
2502              Live by Night
5821                      Gotti
9201                       Made
8864             American Movie
5125              Bulletproof 2
1226    The Godfather: Part III
Name: title, dtype: object